<a href="https://colab.research.google.com/github/Remi-Boutin/DL_project_ENSAE/blob/master/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorboardX

     |████████████████████████████████| 204kB 2.8MB/s 


In [2]:
!git clone https://github.com/Remi-Boutin/DL_project_ENSAE

Cloning into 'DL_project_ENSAE'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 45 (delta 16), reused 8 (delta 3), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [0]:
import time
import os

try:
    os.chdir('DL_project_ENSAE')
except FileNotFoundError:
    if'GANs_tools.py' not in os.listdir(os.getcwd()) :
        print("YOU are not in the right folder")

if not os.path.isdir("Results"):
    os.mkdir('Results')

In [0]:
import sys
import numpy as np
import torch
from torchvision.utils import save_image
from torch import nn, optim
from torch.nn import Sequential
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import torchvision.utils as vutils
import pickle as pkl
from utils import Logger
import pdb
from utils import Logger
import python_utils as utils
from IPython.display import clear_output
import GANs_tools
import matplotlib.pyplot as plt
import importlib
importlib.reload(GANs_tools)
importlib.reload(utils)
from utils import Logger

In [0]:
def images_to_vectors(images):
    return images.view(images.size(0), 784)


def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)


def mnist_data():
    compose = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    out_dir = ''
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

# Data

In [6]:
# Load data
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
# Num batches
num_batches = len(data_loader)

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


# Discriminator

**Reminder :**
 - stride = enjambée = de combien on fait glisser le kernel à chaque calcul
 - padding = rembourage = combien de 1 rajoute-t-on sur les bords de l'image pour combler

In [7]:
class Discriminator(nn.Module):
    def __init__(self, input_shape=1):
        super(Discriminator, self).__init__()
        self.conv1 = Sequential(nn.Conv2d(in_channels=1, out_channels=16, stride=2,
                                          padding=1, kernel_size=5),
                                nn.LeakyReLU(0.1),
                                nn.BatchNorm2d(16))

        self.conv2 = Sequential(nn.Conv2d(in_channels=16, out_channels=16, stride=2, padding=1,
                                          kernel_size=5),
                                nn.LeakyReLU(0.1),
                                nn.BatchNorm2d(16))

        self.hidden = nn.Sequential(torch.nn.Flatten(),
                                    nn.Linear(576, 256),
                                    nn.Sigmoid())

        self.out = nn.Sequential(nn.Linear(256, 1),
                                 nn.Sigmoid())

    def preprocess(self, x):
        return x.view(-1, 1, 28, 28)

    def forward(self, x):
        x = self.preprocess(x)
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        # x = self.conv4(x)
        x = self.hidden(x)
        x = self.out(x)
        return x

In [0]:
class Generator(nn.Module):
    def __init__(self, img_size=28, latent_dim=100, channels=1):
        super(Generator, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.channels = channels
        self.init_size = self.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(self.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh())

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


In [0]:
discriminator = Discriminator()
generator = Generator()
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

discriminator.apply(weights_init)
generator.apply(weights_init)

dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

d_optimizer = optim.Adam(discriminator.parameters(), lr=2e-3)
g_optimizer = optim.Adam(generator.parameters(), lr=2e-3)
loss = nn.BCELoss().cuda()

In [0]:
num_test_samples = 16
num_epochs = 1
logger = Logger(model_name='VGAN', data_name='MNIST')

In [0]:
def fit(num_epochs, data_loader, netG, netD, WGAN_training=True, lower_clamp=-0.05, upper_clamp=0.05):
    if WGAN_training:
        if not os.path.isdir('./Results/images'):
            os.mkdir('./Results/images')
    for epoch in range(num_epochs):
        for n_batch, (real_batch, _) in enumerate(data_loader):

            # 1. Train Discriminator
            #Initialization of gradients
            for p in netD.parameters(): # reset requires_grad
                p.requires_grad = True # they are set to False below in netG update
            netD.zero_grad()
            
            if WGAN_training:
                # Set the parameters in a cube to respect the Lipsichitz condition necessary to the gradient computation
                for p in netD.parameters():
                        p.data.clamp_(lower_clamp, upper_clamp)
                    
            # Generate fake data and noise
            real_data = Variable(images_to_vectors(real_batch)).type(dtype)  
            noise_vector = GANs_tools.noise(real_data.size(0), dtype)
            fake_data = netG(noise_vector).detach().type(dtype)

            # Train the Discriminator
            if WGAN_training:
                d_pred_real, d_pred_fake = GANs_tools.train_discriminator(d_optimizer,
                                                                                   netD, loss,
                                                                                   real_data, fake_data, dtype,
                                                                                   WGAN_training)
                error_D= d_pred_real - d_pred_fake
            
            else:
                d_error, d_pred_real, d_pred_fake = GANs_tools.train_discriminator(d_optimizer,
                                                                                   netD, loss,
                                                                                   real_data, fake_data, dtype,
                                                                                   WGAN_training)

            # 2. Train Generator
            
            #Turn off discriminator gradients traceback
            for p in netD.parameters():
                p.requires_grad = False # to avoid computation
            netG.zero_grad() # Initialization of generator gradients
            
            
            # Generate fake data
            generated_data = netG(GANs_tools.noise(real_batch.size(0), dtype))
            # Train Generator
            if WGAN_training:
                error_G = GANs_tools.train_generator(g_optimizer, netD, loss, generated_data, dtype,WGAN_training)
                
            else:
                g_error = GANs_tools.train_generator(g_optimizer, netD, loss, fake_data, dtype,WGAN_training)
            
            # Log error
            if WGAN_training:
                if (n_batch) % 100 == 0:
                    test_noise = GANs_tools.noise(num_test_samples, dtype)
                    print('[%d/%d][%d/%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
                          % (epoch, num_epochs, n_batch, len(data_loader), 
                             error_D.data[0], error_G.data[0], d_pred_real.data[0], d_pred_fake.data[0]))
                    real_cpu = real_batch.mul(0.5).add(0.5)            
                    vutils.save_image(real_cpu, './Results/images/real_samples_epoch{}.png'.format(epoch))
                    fake = netG(test_noise)
                    fake.data = fake.data.mul(0.5).add(0.5)
                    vutils.save_image(fake.data, './Results/images/fake_epoch{}.png'.format(epoch))
            else:
                logger.log(d_error, g_error, epoch, n_batch, num_batches)

                    # Display Progress
                if (n_batch) % 100 == 0:
                    clear_output(True)
                    # Display Images
                    test_noise = GANs_tools.noise(num_test_samples, dtype)
                    test_images = vectors_to_images(netG(test_noise)).data.cpu()
                    logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches)
                    # Display status Logs
                    logger.display_status(
                        epoch, num_epochs, n_batch, num_batches,
                        d_error, g_error, d_pred_real, d_pred_fake
                    )
                # Model Checkpoints
                logger.save_models(netG, netD, epoch)

In [0]:
importlib.reload(GANs_tools)
importlib.reload(utils)
from utils import Logger

In [15]:
fit(10, data_loader, generator, discriminator, WGAN_training=True, lower_clamp=-0.01, upper_clamp=0.01)

[0/10][0/600] Loss_D: -0.000964 Loss_G: 0.528393 Loss_D_real: 0.527693 Loss_D_fake 0.528656
[0/10][100/600] Loss_D: -0.001425 Loss_G: 0.528588 Loss_D_real: 0.527322 Loss_D_fake 0.528748
[0/10][200/600] Loss_D: -0.001166 Loss_G: 0.528715 Loss_D_real: 0.526901 Loss_D_fake 0.528068
[0/10][300/600] Loss_D: -0.001364 Loss_G: 0.527883 Loss_D_real: 0.527226 Loss_D_fake 0.528590
[0/10][400/600] Loss_D: -0.000441 Loss_G: 0.528284 Loss_D_real: 0.527513 Loss_D_fake 0.527953
[0/10][500/600] Loss_D: -0.000674 Loss_G: 0.528430 Loss_D_real: 0.527904 Loss_D_fake 0.528578
[1/10][0/600] Loss_D: -0.000836 Loss_G: 0.527938 Loss_D_real: 0.527732 Loss_D_fake 0.528568
[1/10][100/600] Loss_D: -0.000119 Loss_G: 0.528187 Loss_D_real: 0.528043 Loss_D_fake 0.528162
[1/10][200/600] Loss_D: -0.000315 Loss_G: 0.528129 Loss_D_real: 0.527885 Loss_D_fake 0.528200
[1/10][300/600] Loss_D: -0.000030 Loss_G: 0.528232 Loss_D_real: 0.527357 Loss_D_fake 0.527388
[1/10][400/600] Loss_D: -0.000805 Loss_G: 0.527527 Loss_D_real: 

In [0]:
def fit_bis(num_epochs, data_loader, netG, netD, n_critic=5, sample_interval=400, WGAN_training=True, lower_clamp=-0.05, upper_clamp=0.05):
    if WGAN_training:
        if not os.path.isdir('./Results/images'):
            os.mkdir('./Results/images')
    batches_done = 0
    for epoch in range(num_epochs):
        for i, (imgs, _)  in enumerate(data_loader):
          z = GANs_tools.noise(imgs.shape[0], dtype)
          # Generate a batch of images
          real_imgs = Variable(imgs.type(dtype))
          generated_imgs = netG(z).detach().type(dtype)
          # Adversarial loss
          loss_D = -torch.mean(netD(real_imgs)) + torch.mean(netD(generated_imgs))
          loss_D.backward()
          d_optimizer.step()

          # Clip weights of discriminator
          for p in netD.parameters():
              p.data.clamp_(lower_clamp, upper_clamp)

          # Train the generator every n_critic iterations
          if i % n_critic == 0:

              # -----------------
              #  Train Generator
              # -----------------

              g_optimizer.zero_grad()

              # Generate a batch of images
              gen_imgs = generator(z)
              # Adversarial loss
              loss_G = -torch.mean(discriminator(gen_imgs))

              loss_G.backward()
              g_optimizer.step()

             

          if batches_done % sample_interval == 0:
            print(
                  "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                  % (epoch, num_epochs, batches_done % len(data_loader), len(data_loader), loss_D.item(), loss_G.item())
              )
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
          batches_done += 1




In [63]:
fit_bis(50, data_loader, generator, discriminator, WGAN_training=True, lower_clamp=-0.01, upper_clamp=0.01)

[Epoch 0/50] [Batch 0/600] [D loss: 0.000018] [G loss: -0.785837]
[Epoch 0/50] [Batch 400/600] [D loss: -0.000078] [G loss: -0.788298]
[Epoch 1/50] [Batch 200/600] [D loss: 0.000020] [G loss: -0.789673]
[Epoch 2/50] [Batch 0/600] [D loss: -0.000196] [G loss: -0.790481]
[Epoch 2/50] [Batch 400/600] [D loss: -0.000400] [G loss: -0.790462]
[Epoch 3/50] [Batch 200/600] [D loss: -0.000339] [G loss: -0.790644]
[Epoch 4/50] [Batch 0/600] [D loss: -0.000120] [G loss: -0.790931]
[Epoch 4/50] [Batch 400/600] [D loss: -0.000046] [G loss: -0.791406]
[Epoch 5/50] [Batch 200/600] [D loss: 0.000026] [G loss: -0.791447]
[Epoch 6/50] [Batch 0/600] [D loss: -0.000026] [G loss: -0.791385]
[Epoch 6/50] [Batch 400/600] [D loss: -0.000035] [G loss: -0.791441]
[Epoch 7/50] [Batch 200/600] [D loss: 0.000041] [G loss: -0.791394]
[Epoch 8/50] [Batch 0/600] [D loss: -0.000012] [G loss: -0.791486]
[Epoch 8/50] [Batch 400/600] [D loss: 0.000057] [G loss: -0.791673]
[Epoch 9/50] [Batch 200/600] [D loss: -0.000050] 

In [53]:
pwd


'/content/DL_project_ENSAE'

In [0]:
os.mkdir('images/')